# 搭建神经网络 笔记
教程视频链接：https://www.bilibili.com/video/BV1hE411t7RN

这篇笔记对应视频合集中的
- 神经网络-搭建小实战和Sequential的使用

pytorch domains官方文档：https://pytorch.org/pytorch-domains

本节里，将运用上一节中的神经网络基本结构搭建神经网络.`torchvision`中提供了很多模型，这里用`CRFAR-10 model`为例。以下是该模型的结构：

![CRFAR10][def]


[def]: ..\images\CRFAR10.png "CRFAR10模型的结构"

In [4]:
import torch
import torchvision

dataset = torchvision.datasets.CIFAR10(root = "../dataset", train=False, download=False, transform=torchvision.transforms.ToTensor())
dataloader = torch.utils.data.DataLoader(dataset=dataset, batch_size=1, drop_last=True)

class Tudui(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, padding=2)
        self.maxpool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = torch.nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, padding=2)
        self.maxpool2 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = torch.nn.Conv2d(kernel_size=5, padding=2, in_channels=32, out_channels=64)
        self.maxpool3 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = torch.nn.Flatten()
        self.linear1 = torch.nn.Linear(in_features=1024, out_features=10)
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.conv3(x)
        x = self.maxpool3(x)
        x = self.flatten(x)
        x = self.linear1(x)
        return x
tudui = Tudui()
for data in dataloader:
    imgs, lables = data
    input = imgs
    output = tudui(input)

使用`torch.nn.Sequantial`可以将神经网络的多部运算打包，减少代码量：

In [5]:
class Tudui2(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.model1 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, padding=2),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, padding=2),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Conv2d(kernel_size=5, padding=2, in_channels=32, out_channels=64),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Flatten(),
            torch.nn.Linear(in_features=1024, out_features=10)
        )
    def forward(self, x):
        x = self.model1(x)
        return x
tudui = Tudui2()

for data in dataloader:
    imgs, lables = data
    input = imgs
    output = tudui(input)

使用`tensorboard`绘制模型也可以达成可视化的效果：

In [ ]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter("../logs")
writer.add_graph(tudui, input)#绘制神经网络的示意图
writer.close()